
# Theory

The problem at hand is that of division:

$$C = \frac{N}{D}$$

The first step to solving this is to split the problem in two: **Reciprocal calculation**, followed by **multiplication**. This is what it looks like:

$$C = N \cdot \left(\frac{1}{D}\right)$$

## Newton-Raphson Method

This is the generic iterative equation according to Newton's method:

$$x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}$$

The idea is to find a function $f(x)$ for which $x = \frac{1}{D}$ is zero. One such function is:

$$f(x) = \frac{1}{x} - D$$

*(Substitute $x = \frac{1}{D}$ in the above equation and it should result in zero)*

Next, we find $f'(x)$ and substitute it in the NM equation to give us an equation that allows successive improvements.

$$x_{i+1} = x_i - \frac{\frac{1}{x_i} - D}{-\frac{1}{x_i^2}} = x_i \cdot (2 - D \cdot x_i)$$

---

## Initial Approximation to the Reciprocal

Here is the final equation for calculating $x_0$, provided $D$ has been scaled to be in the range $[0.5, 1]$:

$$x_0 = \frac{48}{17} - \frac{32}{17}D$$

In numerical algorithms like division, the goal is not necessarily the smallest average error, but guaranteeing the worst-case error ($|\epsilon_0|$) is as small as possible.

We wish to calculate an approximation for the function $1/D$ such that the worst-case error is minimal. The right tool for this is the **Chebyshev Equioscillation Theorem**.

Chebyshev approximation is used because it provides the **Best Uniform Approximation** (or **Minimax Approximation**). This means that out of all possible polynomials of a given degree, the Chebyshev method yields the one that minimizes the maximum absolute error across the entire target interval.

We start by formulating the error function on which equioscillation will be applied. The error function for figuring out the reciprocal $x_0 = 1/D$ using a simple straight line $x_0 = T_0 + T_1 \cdot D$ (a linear equation) tells us how far off our guess is from the perfect answer. Because we want the total result $D \cdot x_0$ to be near $1$, we make the error function $f(D)$ measure the difference between that product and $1$. The formula is $f(D) = 1 - D \cdot x_0$. When we plug in the straight-line guess, the formula becomes:

$$f(D) = 1 - T_0 D - T_1 D^2$$

The main goal is to pick the numbers $T_0$ and $T_1$ that minimize the absolute value of this error $f(D)$ everywhere in the range. This is exactly what the Chebyshev method does.

We need to constrain the values that $D$ can take. Bounding $D$ guarantees that the starting error is small enough for the subsequent iterations to converge quickly and predictably to full fixed-point precision. Without this bound, a much more complex, higher-degree polynomial would be needed, defeating the efficiency goal. We bound $D$ to be $[0.5, 1]$. In code, this scaling can be achieved through simple bit-shifts. As long as we scale the numerator too, the result remains correct.

The theorem states that a polynomial is the best uniform approximation to a continuous function over an interval if and only if the error function alternates between its maximum positive and maximum negative values at least $(n+2)$ times, where $n$ is the degree of the polynomial. Since $n = 1$ (linear approximation), we need $1 + 2 = 3$ alternating extrema: at the two endpoints ($D = 1/2$ and $D = 1$) and the local extremum ($D_{min}$) between them.

The location of the local extremum is found by setting the derivative to zero:

$$f'(D) = -T_0 - 2T_1 D = 0, \quad \text{which gives:} \quad D_{min} = -\frac{T_0}{2T_1}$$

The first condition of the theorem is that the error magnitude is equal at endpoints, i.e., $f(1/2) = f(1)$.

$$1 - \frac{T_0}{2} - \frac{T_1}{4} = 1 - T_0 - T_1$$

This simplifies to:

$$T_0 = -\frac{3}{2}T_1$$

The second condition states that the error at endpoints must be the negative of the error at the extremum, i.e., $f(1) = -f(D_{min})$.

$$1 - T_0 - T_1 = -\left(1 - T_0 D_{min} - T_1 D_{min}^2\right)$$

Substituting $D_{min}$ and simplifying:

$$1 - T_0 - T_1 = -\left(1 + \frac{T_0^2}{4T_1}\right)$$

Substituting the value of $T_0$ from above:

$$1 - \left(-\frac{3}{2}T_1\right) - T_1 = -1 - \frac{\left(-3/2 \cdot T_1\right)^2}{4T_1}$$

$$1 + \frac{1}{2}T_1 = -1 - \frac{9T_1}{16}$$

Solving this linear equation for $T_1$:

$$2 = -\frac{17T_1}{16}$$

$$T_1 = -\frac{32}{17}$$

Substituting $T_1$ back into the original equation to find $T_0$:

$$T_0 = -\frac{3}{2} \cdot \left(-\frac{32}{17}\right)$$

$$T_0 = \frac{48}{17}$$

The resulting linear approximation is $x_0 = T_0 + T_1 D$:

$$x_0 = \frac{48}{17} - \frac{32}{17}D$$

This equation gives the optimal initial estimate for the reciprocal that can be refined by iterations of the Newton-Raphson equation.
```

In [14]:
import numpy as np

In [ ]:
def normalize(D):
    # Dividing or multiplying  by 2 can easily be done by bit shifting
    scale = 1.0
    while(D<0.5 or D>1):
        if D<0.5:
            D = D*2
            scale = scale/2 
        else:
            D = D/2
            scale = scale*2
    return D, scale

def initialGuess(D):
    return 48/17 - (32/17)*D

def x_n_plus_1(x_n,D):
    return x_n*(2 - D*x_n)

def newtonRaphson(d):
    D,scale = normalize(d)
    x_n = initialGuess(D)
    x_n_1 = x_n_plus_1(x_n,D)
    while(abs(x_n_1-x_n)>1e-6):
        x_n = x_n_1
        x_n_1 = x_n_plus_1(x_n,D)
    return x_n_1/scale
def round_to_int(num):
    int_num = int(num) 
    frac = abs(num - int_num) 
    if frac >= 0.5:
        return int_num + 1 if num >= 0 else int_num - 1
    return int_num

def fixedPoint(num, m, n):
  scale = 2**n
  fixed_int = round_to_int(num*scale)
  quantized = fixed_int/scale
  return quantized

In [55]:
arr = [3/21, 7/31, 9/41, 11/51, 17/81, 31/67, 41/91, 51/101, 67/131, 81/151]
numerators = [3, 7, 9, 11, 17, 31, 41, 51, 67, 81]
denominators = [21, 31, 41, 51, 81, 67, 91, 101, 131, 151]

arr_processed = []

for i in range(len(arr)):
    fraction =  newtonRaphson(denominators[i])
    res = numerators[i]*fraction
    arr_processed.append(fixedPoint(res, 8, 8))

arr_processed = np.array(arr_processed)
np.savetxt("arr_processed.txt", arr_processed, fmt='%.8f')


## Finally here I fecth verilog result and compare them

In [ ]:
arr_processed_verilog = np.loadtxt("arr_processed_verilog.txt")

# difference table
print("Python Result\t\tVerilog Result\t\tDifference")
for p, v in zip(arr_processed, arr_processed_verilog):
    print(f"{p:.8f}\t\t{v:.8f}\t\t{abs(p-v):.8f}")


Python Result		Verilog Result		Difference
0.14453125		0.14453125		0.00000000
0.22656250		0.22656250		0.00000000
0.21875000		0.21875000		0.00000000
0.21484375		0.21484375		0.00000000
0.21093750		0.21093750		0.00000000
0.46093750		0.46093750		0.00000000
0.44921875		0.44921875		0.00000000
0.50390625		0.50390625		0.00000000
0.51171875		0.51171875		0.00000000
0.53515625		0.53515625		0.00000000
